# Geodatenanalyse 2: 
# Termin 1 - Datenaufbereitung
### Aufgabe

### Einlesen des Datensatzes und Ersetzen von Werten unter der NWG

Überprüfe zunächst, welche Werte als Platzhalter für NULL-Werte dienen. Falls sie nicht bereits als `NaN` erkannt werden, kannst du sie mithilfe von `pd.read_csv()` und dem Argument `na_values` direkt beim Einlesen des Datensatzes ersetzen.

### Ersetzen von NULL-Werten

Verwende `DataFrame.fillna()` oder `Series.fillna()` zum Ersetzen von NULL-Werten im gesamten DataFrame oder in einer spezifischen Spalte.

### Aggregierung der Daten für jede Messstelle

Nutze `DataFrame.groupby()` für die Aggregierung nach Messstellen.

### Splitting der Daten (85:15)

Verwende `train_test_split` aus `sklearn.model_selection` für das Aufteilen des Datensatzes.

### Standardisierung der numerischen Variablen

Nutze `StandardScaler` aus `sklearn.preprocessing` für die Standardisierung der numerischen Variablen.

### Encoding der Landnutzung

Erstelle eine neue binäre Spalte basierend auf der Landnutzung mit Hilfe von `DataFrame.apply()` und einer entsprechenden Funktion.

### Target-Encoding der Hydrogeologie

Implementiere eine Funktion, die die Nitratkonzentration nach Hydrogeologie gruppiert und die entsprechenden Mittelwerte berechnet. Verwende dann `Series.map()` oder `DataFrame.replace()` für das Target-Encoding.

### Speichern der Ergebnisse

Nutze `DataFrame.to_csv()` für das Speichern des bearbeiteten Datensatzes.


Bereite jetzt selbst den Datensatz "Nitratmessungen_aufgabe.csv" auf:
- Ersetze die Werte unter der NWG 
- Ersetze alle NULL-Values (bzw. Platzhalter für NaN)
- Aggregiere die Daten für jede Messstelle
- Splitte die Date (85:15)
- Standardisiere die numerischen Variablen
- Encode die Landnutzung binär nach der Landwirtschaft (Corine = 2XX) --> vorhanden = 1, nicht vorhanden = 0
- Target-Encode die Hyrogeologie auf Basis der Nitratkonzentration
- Speichere deine Ergebnisse
- Fertig! :)

Tipps:
- für die Features der Sauerstoff-Konzentration und der Hydrogeologie liegen die NULL-Values nicht als diese direkt vor, hier ist es hilfreich mal die Features mit ".describe()" bzw. ".unique()" zu betrachten

#### Laden der Bibliotheken

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



#### Nitratdatensatz:

In [2]:
#Mögliche Fehler: encoding falsch, sep falsch, index_columns falsch gewählt 
data1 = pd.read_csv('Nitratmessungen_aufgabe.csv', sep=';', encoding="ISO-8859-1")
data1.head()

,Messstelle,GW-Nummer,Datum,NO3 [mg/l],O2 [mg/l],RASTERVALU,HYDROGEOL3
0,"BBR Betonwerk, Umkirch",918/069-2,11.10.2006 15:20,"9,4","2,4",121,Quartäre Kiese und Sande (GWL)
1,"BBR Betonwerk, Umkirch",918/069-2,17.09.2007 14:20,12,"2,1",121,Quartäre Kiese und Sande (GWL)
2,"BBR Betonwerk, Umkirch",918/069-2,23.09.2008 08:05,"7,6","2,5",121,Quartäre Kiese und Sande (GWL)
3,"BBR Betonwerk, Umkirch",918/069-2,10.11.2008 09:10,"12,2","1,2",121,Quartäre Kiese und Sande (GWL)
4,"BBR Betonwerk, Umkirch",918/069-2,09.09.2009 14:08,12,"1,3",121,Quartäre Kiese und Sande (GWL)


#### Filtern des Datensatzes

In [3]:
data = data1[['Messstelle','GW-Nummer','Datum', 'NO3 [mg/l]','O2 [mg/l]','RASTERVALU', 'HYDROGEOL3']]
data = data.assign(Datum=pd.to_datetime(data['Datum'], format='%d.%m.%Y %H:%M'))
data.head()

,Messstelle,GW-Nummer,Datum,NO3 [mg/l],O2 [mg/l],RASTERVALU,HYDROGEOL3
0,"BBR Betonwerk, Umkirch",918/069-2,2006-10-11 15:20:00,"9,4","2,4",121,Quartäre Kiese und Sande (GWL)
1,"BBR Betonwerk, Umkirch",918/069-2,2007-09-17 14:20:00,12,"2,1",121,Quartäre Kiese und Sande (GWL)
2,"BBR Betonwerk, Umkirch",918/069-2,2008-09-23 08:05:00,"7,6","2,5",121,Quartäre Kiese und Sande (GWL)
3,"BBR Betonwerk, Umkirch",918/069-2,2008-11-10 09:10:00,"12,2","1,2",121,Quartäre Kiese und Sande (GWL)
4,"BBR Betonwerk, Umkirch",918/069-2,2009-09-09 14:08:00,12,"1,3",121,Quartäre Kiese und Sande (GWL)


## 1. Ersetze die Werte unter der NWG 

In [4]:
# Funktion um Werte unter der NWG durch 0.5*NWG zu ersetzen
def u_NWG(X):
    # Wenn X eine String ist, wird er in eine float umgewandelt.
    if isinstance(X, str):
        # Überprüfung, ob der String ein "<" enthält, der auf eine Bereichsangabe hinweist.
        if '<' in X:
            # Extrahiere den Bereichswert und dividiere ihn durch 2.
            Y = float(X.split('<')[1].replace(',', '.'))/2
        else:
            # Wenn kein "<" vorhanden ist, wird der String in einen float umgewandelt.
            Y = float(X.replace(',', '.'))
    else:
        # Wenn X kein String ist, wird der Wert einfach zugewiesen.
        Y = X   
    # Gib den finalen Wert zurück.
    return Y

In [5]:
data = data.assign(O2= data['O2 [mg/l]'].apply(u_NWG),
           NO3 = data['NO3 [mg/l]'].apply(u_NWG))
data.drop(columns=['NO3 [mg/l]','O2 [mg/l]'], inplace=True)
data.sort_values("NO3", ascending=True).head()

,Messstelle,GW-Nummer,Datum,RASTERVALU,HYDROGEOL3,O2,NO3
7987,"GWM 3 Firma CU Chemie Uetikon, Lahr",374/066-7,2007-09-12 11:55:00,121,Quartäre Kiese und Sande (GWL),0.3,0.05
4099,"GWM E21/1 ZWK Kurpfalzwalldorf, Walldorf",22/306-3,2018-03-13 15:50:00,313,Quartäre Kiese und Sande (GWL),0.4,0.05
4614,"GWM Flach (1960), Auenheim",2010/113-0,2007-09-14 13:10:00,211,Quartäre Kiese und Sande (GWL),0.3,0.05
3792,"GWM B2 Flach, Neumühl",2006/114-9,2007-09-14 10:45:00,211,Quartäre Kiese und Sande (GWL),0.3,0.05
2852,"BR 5 Südzucker, Waghäusel",244/307-5,2007-09-20 16:03:00,121,Quartäre Kiese und Sande (GWL),0.1,0.05


In [6]:
print(data.NO3.min())
print(data.O2.min())

0.05
-999.0


## 2. Ersetze alle NULL-Values (bzw. Platzhalter für NaN)

In [7]:
#Platzhalter zuerst durch Nan ersetzen und anschließend NaN durch Mittelwert ersetzen.
#Ansonsten wird der Mittelwert falsch berechnet

In [8]:
data['O2'].mean()

-35.729352110362946

### 2.1 Ersetze Platzhalter

In [9]:
data['O2'] = data['O2'].replace(-999.0, None)

data['O2'].mean()

3.3677646757241546

In [10]:
data['HYDROGEOL3'] = data['HYDROGEOL3'].replace("-", None)
data['HYDROGEOL3'].unique()

array(['Quartäre Kiese und Sande (GWL)', 'Oberer Muschelkalk (GWL)',
       'Paläozoikum, Kristallin (GWG)', None,
       'Unterjura und Mitteljura (GWG)',
       'Tertiär im Oberrheingraben (GWG)',
       'Oberjura (Raurasische Fazies) (GWL)'], dtype=object)

### 2.2 Ersetzen von fehlenden Werte (NULL-Values)


Kontrolle der NULL-Values:

In [11]:
data.isnull().sum()

Messstelle      0
GW-Nummer       0
Datum           0
RASTERVALU      0
HYDROGEOL3     56
O2            475
NO3            20
dtype: int64

#### 2.2.1 Nan in numerische Spalten

In [12]:
# Fill missing values in 'O2' and 'NO3' columns with their means
data['O2'] = data['O2'].fillna(data['O2'].mean())
data['NO3'] = data['NO3'].fillna(data['NO3'].mean())

data.head()

C:\Users\Marc Ohmer\AppData\Local\Temp\ipykernel_26584\2438344789.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['O2'] = data['O2'].fillna(data['O2'].mean())


,Messstelle,GW-Nummer,Datum,RASTERVALU,HYDROGEOL3,O2,NO3
0,"BBR Betonwerk, Umkirch",918/069-2,2006-10-11 15:20:00,121,Quartäre Kiese und Sande (GWL),2.4,9.4
1,"BBR Betonwerk, Umkirch",918/069-2,2007-09-17 14:20:00,121,Quartäre Kiese und Sande (GWL),2.1,12.0
2,"BBR Betonwerk, Umkirch",918/069-2,2008-09-23 08:05:00,121,Quartäre Kiese und Sande (GWL),2.5,7.6
3,"BBR Betonwerk, Umkirch",918/069-2,2008-11-10 09:10:00,121,Quartäre Kiese und Sande (GWL),1.2,12.2
4,"BBR Betonwerk, Umkirch",918/069-2,2009-09-09 14:08:00,121,Quartäre Kiese und Sande (GWL),1.3,12.0


#### 2.2.1 NAN in kategorische Spalten


In [13]:
hy_mode = data.HYDROGEOL3.mode()[0]
print(data.HYDROGEOL3.value_counts(),'\n\n',
      'Modus:',
      hy_mode)

HYDROGEOL3
Quartäre Kiese und Sande (GWL)         11955
Paläozoikum, Kristallin (GWG)             50
Tertiär im Oberrheingraben (GWG)          37
Oberjura (Raurasische Fazies) (GWL)       35
Oberer Muschelkalk (GWL)                  26
Unterjura und Mitteljura (GWG)            19
Name: count, dtype: int64 

 Modus: Quartäre Kiese und Sande (GWL)


In [14]:
data = data.assign(Hydrogeologie = data.loc[:,'HYDROGEOL3'].fillna(hy_mode))
data.drop(columns='HYDROGEOL3', inplace=True)

#### Kontrolle

In [15]:
data.isnull().sum()

Messstelle       0
GW-Nummer        0
Datum            0
RASTERVALU       0
O2               0
NO3              0
Hydrogeologie    0
dtype: int64

## 3. Aggregiere die Daten für jede Messstelle

In [16]:
messstellen1 = data[['Messstelle','NO3','O2']].groupby('Messstelle').mean()
messstellen2 = data[['Messstelle','Hydrogeologie','RASTERVALU']].groupby('Messstelle').agg(pd.Series.mode)

messstellen1.reset_index(inplace=True)
messstellen2.reset_index(inplace=True)

In [17]:
Messstellen  = messstellen1.merge(messstellen2, on='Messstelle')
Messstellen.head()

,Messstelle,NO3,O2,Hydrogeologie,RASTERVALU
0,"BBR 1 Firma Schultis, Riegel",6.259259,1.315473,Quartäre Kiese und Sande (GWL),121
1,"BBR 2 Firma Thieme, Teningen",1.452941,0.823723,Quartäre Kiese und Sande (GWL),121
2,"BBR 2 Kronenwiese Firma Burda Werk 1, Offenburg",5.600000,1.984615,Quartäre Kiese und Sande (GWL),121
3,"BBR 3186 im Garten der alten Schule, Langhurst",6.968750,0.730346,Quartäre Kiese und Sande (GWL),112
4,"BBR 998 A Kehlerstrasse, Neuried-Auenheim",56.164286,1.116071,Quartäre Kiese und Sande (GWL),112


## 4. Splitte die Date (85:15)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
train, test = train_test_split(Messstellen, test_size=0.85, random_state=43)

In [20]:
print('Größe Trainingsdaten: ', train.shape)
print('Größe Testdaten: ', test.shape)
print('Größe Testdaten in %: ',(len(train)/len(test)*100))

Größe Trainingsdaten:  (76, 5)
Größe Testdaten:  (431, 5)
Größe Testdaten in %:  17.633410672853827


## 5. Standardisiere die numerischen Variablen

### 5.1 Normieren

In [21]:
from sklearn.preprocessing import MinMaxScaler

In [22]:
scaler = MinMaxScaler()
scaler.fit(train[['NO3']])

MinMaxScaler()

In [23]:
train['NO3_norm']= scaler.transform(train[['NO3']])
test['NO3_norm'] = scaler.transform(test[['NO3']])

In [24]:
print('X_min: ', train.NO3_norm.min())
print('X_max: ', train.NO3_norm.max())
print('X_max: ', train.NO3_norm.mean())

X_min:  0.0
X_max:  1.0
X_max:  0.2139923749848081


### 5.1 Standardisieren

In [25]:
from sklearn.preprocessing import StandardScaler

In [26]:
scaler = StandardScaler()
scaler.fit(train[['NO3']])

StandardScaler()

In [27]:
train['NO3_std']= scaler.transform(train[['NO3']])
test['NO3_std'] = scaler.transform(test[['NO3']])

In [28]:
print('mean(X): ', train.NO3_std.mean())
print('std(X): ', train.NO3_std.std())

mean(X):  1.7383755253999163e-16
std(X):  1.0066445913694333


In [29]:
train.head()

,Messstelle,NO3,O2,Hydrogeologie,RASTERVALU,NO3_norm,NO3_std
499,"TB Wagrain, Forchheim",49.671875,4.228125,Quartäre Kiese und Sande (GWL),211,0.470214,1.032569
445,"TB 4 WV, Karlsdorf Neuthard",0.405357,0.661349,Quartäre Kiese und Sande (GWL),231,0.001629,-0.855822
99,"BR NB 22 Eichelgarten, KA-Rüppurr",29.650000,3.570000,Quartäre Kiese und Sande (GWL),311,0.279781,0.265128
61,"BR FLB2 Steinmauern, Rastatt",4.700000,1.047081,Quartäre Kiese und Sande (GWL),211,0.042476,-0.691208
7,"BBR Fa. Hellma GmbH, Müllheim",32.135714,6.510714,Quartäre Kiese und Sande (GWL),121,0.303423,0.360406


In [30]:
test.head()

,Messstelle,NO3,O2,Hydrogeologie,RASTERVALU,NO3_norm,NO3_std
498,"TB WW Neuburgweier, Rheinstetten",0.664062,1.625000,Quartäre Kiese und Sande (GWL),231,0.004090,-0.845906
328,"GWM K 20 Kehl, Goldscheuer",0.227778,0.255556,Quartäre Kiese und Sande (GWL),231,-0.000060,-0.862629
321,"GWM G0/1 KIT, Leopoldshafen",6.945455,3.259091,Quartäre Kiese und Sande (GWL),313,0.063833,-0.605139
170,"GWM 1279 A, Oberhausen",14.138462,7.215385,Quartäre Kiese und Sande (GWL),231,0.132248,-0.329431
432,"TB 116 Universität Albertstrasse, Freiburg",14.133333,7.412369,Quartäre Kiese und Sande (GWL),121,0.132199,-0.329627


## 6. Encode die Landnutzung binär nach der Landwirtschaft (Corine = 2XX) &rarr; vorhanden = 1, nicht vorhanden = 0

In [31]:
train['Corine'] = train['RASTERVALU'].apply(lambda x: int(str(x)[0]))
train['Artificial_Surface'] = train['Corine'].apply(lambda x: 1 if x == 2  else 0)

test['Corine'] = test['RASTERVALU'].apply(lambda x: int(str(x)[0]))
test['Artificial_Surface'] = test['Corine'].apply(lambda x: 1 if x == 2  else 0)

In [32]:
#train.drop(columns='RASTERVALU', inplace=True)
#test.drop(columns='RASTERVALU', inplace=True)

In [33]:
train

,Messstelle,NO3,O2,Hydrogeologie,RASTERVALU,NO3_norm,NO3_std,Corine,Artificial_Surface
499,"TB Wagrain, Forchheim",49.671875,4.228125,Quartäre Kiese und Sande (GWL),211,0.470214,1.032569,2,1
445,"TB 4 WV, Karlsdorf Neuthard",0.405357,0.661349,Quartäre Kiese und Sande (GWL),231,0.001629,-0.855822,2,1
99,"BR NB 22 Eichelgarten, KA-Rüppurr",29.650000,3.570000,Quartäre Kiese und Sande (GWL),311,0.279781,0.265128,3,0
61,"BR FLB2 Steinmauern, Rastatt",4.700000,1.047081,Quartäre Kiese und Sande (GWL),211,0.042476,-0.691208,2,1
7,"BBR Fa. Hellma GmbH, Müllheim",32.135714,6.510714,Quartäre Kiese und Sande (GWL),121,0.303423,0.360406,1,0
...,...,...,...,...,...,...,...,...,...
277,"GWM B2 Tief Kieswerk Peter, Honau",0.958333,1.008333,Quartäre Kiese und Sande (GWL),512,0.006888,-0.834626,5,0
305,"GWM F Sportzentrum, Schriesheim",5.061667,5.246667,Quartäre Kiese und Sande (GWL),112,0.045916,-0.677345,1,0
255,"GWM 6, Neuried Dundenheim",74.896429,3.650118,Quartäre Kiese und Sande (GWL),211,0.710130,1.999428,2,1
320,"GWM Friedrichstal, Friedrichstal",4.788462,1.014271,Quartäre Kiese und Sande (GWL),211,0.043318,-0.687817,2,1


## 7. Target-Encode die Hyrogeologie auf Basis der Nitratkonzentration

In [34]:
from category_encoders.target_encoder import TargetEncoder

In [35]:
encoder = TargetEncoder()
train['Target_Hy'] = encoder.fit_transform(train['Hydrogeologie'],train['NO3'])
test['Target_Hy'] = encoder.transform(test['Hydrogeologie'])
train.head()

,Messstelle,NO3,O2,Hydrogeologie,RASTERVALU,NO3_norm,NO3_std,Corine,Artificial_Surface,Target_Hy
499,"TB Wagrain, Forchheim",49.671875,4.228125,Quartäre Kiese und Sande (GWL),211,0.470214,1.032569,2,1,22.995068
445,"TB 4 WV, Karlsdorf Neuthard",0.405357,0.661349,Quartäre Kiese und Sande (GWL),231,0.001629,-0.855822,2,1,22.995068
99,"BR NB 22 Eichelgarten, KA-Rüppurr",29.650000,3.570000,Quartäre Kiese und Sande (GWL),311,0.279781,0.265128,3,0,22.995068
61,"BR FLB2 Steinmauern, Rastatt",4.700000,1.047081,Quartäre Kiese und Sande (GWL),211,0.042476,-0.691208,2,1,22.995068
7,"BBR Fa. Hellma GmbH, Müllheim",32.135714,6.510714,Quartäre Kiese und Sande (GWL),121,0.303423,0.360406,1,0,22.995068


## 8. Speichere deine Ergebnisse

In [36]:
#train.to_csv('Aufg_1_train.csv')
#test.to_csv('Aufg_1_test.csv')

## Fertig